<a href="https://colab.research.google.com/github/adealide42ax/CoLab/blob/main/colabs/batch_variant_scoring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Batch variant scoring

In this notebook, we demonstrate how to score many variants using AlphaGenome.

To prepare numerous variants for scoring, provide the following information as
columns in a tab-separated Variant Call Format (VCF) file: - `variant_id`: A
unique identifier for each variant. - `CHROM`: Chromosome, specified as a string
beginning with 'chr' (e.g., 'chr1'). - `POS`: Integer representing the base pair
position (1-based; build hg38 (human) or mm10 (mouse) - see
[FAQ](https://www.alphagenomedocs.com/faqs.html#how-do-i-define-a-variant)). -
`REF`: The reference nucleotide sequence at the specified position. - `ALT`: The
alternate nucleotide sequence at the specified position.

``` {tip}
Open this tutorial in Google colab for interactive viewing.
```

In [2]:
# @title Install AlphaGenome

# @markdown Run this cell to install AlphaGenome.
from IPython.display import clear_output
! pip install alphagenome
clear_output()

In [3]:
# @title Setup and imports.

from io import StringIO
from alphagenome import colab_utils
from alphagenome.data import genome
from alphagenome.models import dna_client, variant_scorers
from google.colab import data_table, files
import pandas as pd
from tqdm import tqdm

data_table.enable_dataframe_formatter()

# Load the model.
dna_model = dna_client.create(colab_utils.get_api_key())

In [8]:
vcf_file = '/content/KQ_mutations.vcf'
vcf = pd.read_csv(vcf_file, sep='\t')
vcf

,variant_id,CHROM,POS,REF,ALT
0,chr1_16152382_C_T_b38,chr1,16152382,C,T
1,chr4_186659580_G_C_b38,chr4,186659580,G,C
2,chr4_186659586_C_T_b38,chr4,186659586,C,T
3,chr5_87333291_C_T_b38,chr5,87333291,C,T
4,chr9_136535031_C_T_b38,chr9,136535031,C,T
5,chr11_9584203_A_G_b38,chr11,9584203,A,G
6,chr17_7674180_C_T_b38,chr17,7674180,C,T
7,chr5_87285240_A_G_b38,chr5,87285240,A,G
8,chr5_87389496_G_A_b38,chr5,87389496,G,A
9,chr17_7674218_T_C_b38,chr17,7674218,T,C


In [9]:
# @title Score batch of variants.

# Load VCF file containing variants.
vcf_file = '/content/KQ_mutations.vcf'  # @param
vcf = pd.read_csv(vcf_file, sep='\t')

required_columns = ['variant_id', 'CHROM', 'POS', 'REF', 'ALT']
for column in required_columns:
  if column not in vcf.columns:
    raise ValueError(f'VCF file is missing required column: {column}.')

organism = 'human'  # @param ["human", "mouse"] {type:"string"}

# @markdown Specify length of sequence around variants to predict:
sequence_length = '1MB'  # @param ["16KB", "100KB", "500KB", "1MB"] { type:"string" }
sequence_length = dna_client.SUPPORTED_SEQUENCE_LENGTHS[
    f'SEQUENCE_LENGTH_{sequence_length}'
]

# @markdown Specify which scorers to use to score your variants:
score_rna_seq = True  # @param { type: "boolean"}
score_cage = True  # @param { type: "boolean" }
score_procap = True  # @param { type: "boolean" }
score_atac = True  # @param { type: "boolean" }
score_dnase = True  # @param { type: "boolean" }
score_chip_histone = True  # @param { type: "boolean" }
score_chip_tf = True  # @param { type: "boolean" }
score_polyadenylation = True  # @param { type: "boolean" }
score_splice_sites = True  # @param { type: "boolean" }
score_splice_site_usage = True  # @param { type: "boolean" }
score_splice_junctions = True  # @param { type: "boolean" }

# @markdown Other settings:
download_predictions = False  # @param { type: "boolean" }

# Parse organism specification.
organism_map = {
    'human': dna_client.Organism.HOMO_SAPIENS,
    'mouse': dna_client.Organism.MUS_MUSCULUS,
}
organism = organism_map[organism]

# Parse scorer specification.
scorer_selections = {
    'rna_seq': score_rna_seq,
    'cage': score_cage,
    'procap': score_procap,
    'atac': score_atac,
    'dnase': score_dnase,
    'chip_histone': score_chip_histone,
    'chip_tf': score_chip_tf,
    'polyadenylation': score_polyadenylation,
    'splice_sites': score_splice_sites,
    'splice_site_usage': score_splice_site_usage,
    'splice_junctions': score_splice_junctions,
}

all_scorers = variant_scorers.RECOMMENDED_VARIANT_SCORERS
selected_scorers = [
    all_scorers[key]
    for key in all_scorers
    if scorer_selections.get(key.lower(), False)
]

# Remove any scorers or output types that are not supported for the chosen organism.
unsupported_scorers = [
    scorer
    for scorer in selected_scorers
    if (
        organism.value
        not in variant_scorers.SUPPORTED_ORGANISMS[scorer.base_variant_scorer]
    )
    | (
        (scorer.requested_output == dna_client.OutputType.PROCAP)
        & (organism == dna_client.Organism.MUS_MUSCULUS)
    )
]
if len(unsupported_scorers) > 0:
  print(
      f'Excluding {unsupported_scorers} scorers as they are not supported for'
      f' {organism}.'
  )
  for unsupported_scorer in unsupported_scorers:
    selected_scorers.remove(unsupported_scorer)


# Score variants in the VCF file.
results = []

for i, vcf_row in tqdm(vcf.iterrows(), total=len(vcf)):
  variant = genome.Variant(
      chromosome=str(vcf_row.CHROM),
      position=int(vcf_row.POS),
      reference_bases=vcf_row.REF,
      alternate_bases=vcf_row.ALT,
      name=vcf_row.variant_id,
  )
  interval = variant.reference_interval.resize(sequence_length)

  variant_scores = dna_model.score_variant(
      interval=interval,
      variant=variant,
      variant_scorers=selected_scorers,
      organism=organism,
  )
  results.append(variant_scores)

df_scores = variant_scorers.tidy_scores(results)

if download_predictions:
  df_scores.to_csv('variant_scores.csv', index=False)
  files.download('variant_scores.csv')

df_scores

100%|██████████| 17/17 [00:40<00:00,  2.36s/it]


,variant_id,scored_interval,gene_id,gene_name,gene_type,gene_strand,junction_Start,junction_End,output_type,variant_scorer,...,biosample_type,biosample_life_stage,data_source,endedness,genetically_modified,transcription_factor,histone_mark,gtex_tissue,raw_score,quantile_score
0,chr1:16152382:C>T,chr1:15628094-16676670:.,None,None,None,None,None,None,ATAC,"CenterMaskScorer(requested_output=ATAC, width=...",...,primary_cell,adult,encode,paired,False,NaN,NaN,NaN,0.002438,0.034594
1,chr1:16152382:C>T,chr1:15628094-16676670:.,None,None,None,None,None,None,ATAC,"CenterMaskScorer(requested_output=ATAC, width=...",...,in_vitro_differentiated_cells,adult,encode,paired,False,NaN,NaN,NaN,-0.056592,-0.772583
2,chr1:16152382:C>T,chr1:15628094-16676670:.,None,None,None,None,None,None,ATAC,"CenterMaskScorer(requested_output=ATAC, width=...",...,primary_cell,adult,encode,paired,False,NaN,NaN,NaN,-0.023185,-0.680793
3,chr1:16152382:C>T,chr1:15628094-16676670:.,None,None,None,None,None,None,ATAC,"CenterMaskScorer(requested_output=ATAC, width=...",...,primary_cell,adult,encode,paired,False,NaN,NaN,NaN,-0.003629,-0.226711
4,chr1:16152382:C>T,chr1:15628094-16676670:.,None,None,None,None,None,None,ATAC,"CenterMaskScorer(requested_output=ATAC, width=...",...,primary_cell,adult,encode,paired,False,NaN,NaN,NaN,0.009233,0.291254
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
467981,chr17:7675994:C>T,chr17:7151706-8200282:.,ENSG00000141510,TP53,protein_coding,-,None,None,RNA_SEQ,PolyadenylationScorer(),...,tissue,embryonic,encode,single,False,NaN,NaN,,0.023982,0.987390
467982,chr17:7675994:C>T,chr17:7151706-8200282:.,ENSG00000141510,TP53,protein_coding,-,None,None,RNA_SEQ,PolyadenylationScorer(),...,tissue,embryonic,encode,single,False,NaN,NaN,,0.025155,0.987674
467983,chr17:7675994:C>T,chr17:7151706-8200282:.,ENSG00000141510,TP53,protein_coding,-,None,None,RNA_SEQ,PolyadenylationScorer(),...,tissue,embryonic,encode,single,False,NaN,NaN,,0.022720,0.987100
467984,chr17:7675994:C>T,chr17:7151706-8200282:.,ENSG00000141510,TP53,protein_coding,-,None,None,RNA_SEQ,PolyadenylationScorer(),...,tissue,embryonic,encode,single,False,NaN,NaN,,0.027162,0.987390


Note that the resulting output dataframe could be quite large, especially if you
use many scorers for scoring. Very large dataframes can't be filtered
interactively, but you can interact with them using the standard `pandas`
commands:

In [10]:
# Examine just the effects of the variants on T-cells.
columns = [c for c in df_scores.columns if c != 'ontology_curie']
df_scores[(df_scores['ontology_curie'] == 'UBERON:0001723')][columns]

,variant_id,scored_interval,gene_id,gene_name,gene_type,gene_strand,junction_Start,junction_End,output_type,variant_scorer,...,biosample_type,biosample_life_stage,data_source,endedness,genetically_modified,transcription_factor,histone_mark,gtex_tissue,raw_score,quantile_score
411,chr1:16152382:C>T,chr1:15628094-16676670:.,None,None,None,None,None,None,DNASE,"CenterMaskScorer(requested_output=DNASE, width...",...,tissue,embryonic,encode,paired,False,NaN,NaN,NaN,-2.680111e-02,-0.698927
3411,chr1:16152382:C>T,chr1:15628094-16676670:.,None,None,None,None,None,None,CAGE,"CenterMaskScorer(requested_output=CAGE, width=...",...,tissue,NaN,fantom,NaN,NaN,NaN,NaN,NaN,-3.897476e-02,-0.920450
3684,chr1:16152382:C>T,chr1:15628094-16676670:.,None,None,None,None,None,None,CAGE,"CenterMaskScorer(requested_output=CAGE, width=...",...,tissue,NaN,fantom,NaN,NaN,NaN,NaN,NaN,1.607215e-01,0.990355
9406,chr1:16152382:C>T,chr1:15628094-16676670:.,ENSG00000055070,SZRD1,protein_coding,+,None,None,RNA_SEQ,GeneMaskLFCScorer(requested_output=RNA_SEQ),...,tissue,embryonic,encode,paired,False,NaN,NaN,,-1.032948e-03,-0.710539
9407,chr1:16152382:C>T,chr1:15628094-16676670:.,ENSG00000065526,SPEN,protein_coding,+,None,None,RNA_SEQ,GeneMaskLFCScorer(requested_output=RNA_SEQ),...,tissue,embryonic,encode,paired,False,NaN,NaN,,-1.163840e-03,-0.737944
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
449750,chr17:7675994:C>T,chr17:7151706-8200282:.,ENSG00000288993,ENSG00000288993,lncRNA,-,None,None,RNA_SEQ,GeneMaskLFCScorer(requested_output=RNA_SEQ),...,tissue,embryonic,encode,paired,False,NaN,NaN,,-4.768372e-07,-0.011535
466749,chr17:7675994:C>T,chr17:7151706-8200282:.,ENSG00000141510,TP53,protein_coding,-,None,None,SPLICE_SITE_USAGE,GeneMaskSplicingScorer(requested_output=SPLICE...,...,tissue,embryonic,encode,NaN,NaN,NaN,NaN,,8.227539e-01,0.999990
467376,chr17:7675994:C>T,chr17:7151706-8200282:.,ENSG00000141510,TP53,protein_coding,-,7675215,7675993,SPLICE_JUNCTIONS,SpliceJunctionScorer(),...,tissue,embryonic,encode,NaN,NaN,NaN,NaN,,1.333984e+00,0.999710
467377,chr17:7675994:C>T,chr17:7151706-8200282:.,ENSG00000141510,TP53,protein_coding,-,7675236,7675993,SPLICE_JUNCTIONS,SpliceJunctionScorer(),...,tissue,embryonic,encode,NaN,NaN,NaN,NaN,,2.960938e+00,0.999977


In [14]:
df_scores[df_scores['ontology_curie'] == 'UBERON:0001723']

,variant_id,scored_interval,gene_id,gene_name,gene_type,gene_strand,junction_Start,junction_End,output_type,variant_scorer,...,biosample_type,biosample_life_stage,data_source,endedness,genetically_modified,transcription_factor,histone_mark,gtex_tissue,raw_score,quantile_score
411,chr1:16152382:C>T,chr1:15628094-16676670:.,None,None,None,None,None,None,DNASE,"CenterMaskScorer(requested_output=DNASE, width...",...,tissue,embryonic,encode,paired,False,NaN,NaN,NaN,-2.680111e-02,-0.698927
3411,chr1:16152382:C>T,chr1:15628094-16676670:.,None,None,None,None,None,None,CAGE,"CenterMaskScorer(requested_output=CAGE, width=...",...,tissue,NaN,fantom,NaN,NaN,NaN,NaN,NaN,-3.897476e-02,-0.920450
3684,chr1:16152382:C>T,chr1:15628094-16676670:.,None,None,None,None,None,None,CAGE,"CenterMaskScorer(requested_output=CAGE, width=...",...,tissue,NaN,fantom,NaN,NaN,NaN,NaN,NaN,1.607215e-01,0.990355
9406,chr1:16152382:C>T,chr1:15628094-16676670:.,ENSG00000055070,SZRD1,protein_coding,+,None,None,RNA_SEQ,GeneMaskLFCScorer(requested_output=RNA_SEQ),...,tissue,embryonic,encode,paired,False,NaN,NaN,,-1.032948e-03,-0.710539
9407,chr1:16152382:C>T,chr1:15628094-16676670:.,ENSG00000065526,SPEN,protein_coding,+,None,None,RNA_SEQ,GeneMaskLFCScorer(requested_output=RNA_SEQ),...,tissue,embryonic,encode,paired,False,NaN,NaN,,-1.163840e-03,-0.737944
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
449750,chr17:7675994:C>T,chr17:7151706-8200282:.,ENSG00000288993,ENSG00000288993,lncRNA,-,None,None,RNA_SEQ,GeneMaskLFCScorer(requested_output=RNA_SEQ),...,tissue,embryonic,encode,paired,False,NaN,NaN,,-4.768372e-07,-0.011535
466749,chr17:7675994:C>T,chr17:7151706-8200282:.,ENSG00000141510,TP53,protein_coding,-,None,None,SPLICE_SITE_USAGE,GeneMaskSplicingScorer(requested_output=SPLICE...,...,tissue,embryonic,encode,NaN,NaN,NaN,NaN,,8.227539e-01,0.999990
467376,chr17:7675994:C>T,chr17:7151706-8200282:.,ENSG00000141510,TP53,protein_coding,-,7675215,7675993,SPLICE_JUNCTIONS,SpliceJunctionScorer(),...,tissue,embryonic,encode,NaN,NaN,NaN,NaN,,1.333984e+00,0.999710
467377,chr17:7675994:C>T,chr17:7151706-8200282:.,ENSG00000141510,TP53,protein_coding,-,7675236,7675993,SPLICE_JUNCTIONS,SpliceJunctionScorer(),...,tissue,embryonic,encode,NaN,NaN,NaN,NaN,,2.960938e+00,0.999977


In [16]:
df_scores[df_scores['ontology_curie'] == 'UBERON:0001723'].to_csv('variant_scores.csv', index=False)
files.download('variant_scores.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [18]:
# @title Score batch of variants.

# Load VCF file containing variants.
vcf_file = '/content/TJ_mutations.vcf'  # @param
vcf = pd.read_csv(vcf_file, sep='\t')

required_columns = ['variant_id', 'CHROM', 'POS', 'REF', 'ALT']
for column in required_columns:
  if column not in vcf.columns:
    raise ValueError(f'VCF file is missing required column: {column}.')

organism = 'human'  # @param ["human", "mouse"] {type:"string"}

# @markdown Specify length of sequence around variants to predict:
sequence_length = '1MB'  # @param ["16KB", "100KB", "500KB", "1MB"] { type:"string" }
sequence_length = dna_client.SUPPORTED_SEQUENCE_LENGTHS[
    f'SEQUENCE_LENGTH_{sequence_length}'
]

# @markdown Specify which scorers to use to score your variants:
score_rna_seq = True  # @param { type: "boolean"}
score_cage = True  # @param { type: "boolean" }
score_procap = True  # @param { type: "boolean" }
score_atac = True  # @param { type: "boolean" }
score_dnase = True  # @param { type: "boolean" }
score_chip_histone = True  # @param { type: "boolean" }
score_chip_tf = True  # @param { type: "boolean" }
score_polyadenylation = True  # @param { type: "boolean" }
score_splice_sites = True  # @param { type: "boolean" }
score_splice_site_usage = True  # @param { type: "boolean" }
score_splice_junctions = True  # @param { type: "boolean" }

# @markdown Other settings:
download_predictions = False  # @param { type: "boolean" }

# Parse organism specification.
organism_map = {
    'human': dna_client.Organism.HOMO_SAPIENS,
    'mouse': dna_client.Organism.MUS_MUSCULUS,
}
organism = organism_map[organism]

# Parse scorer specification.
scorer_selections = {
    'rna_seq': score_rna_seq,
    'cage': score_cage,
    'procap': score_procap,
    'atac': score_atac,
    'dnase': score_dnase,
    'chip_histone': score_chip_histone,
    'chip_tf': score_chip_tf,
    'polyadenylation': score_polyadenylation,
    'splice_sites': score_splice_sites,
    'splice_site_usage': score_splice_site_usage,
    'splice_junctions': score_splice_junctions,
}

all_scorers = variant_scorers.RECOMMENDED_VARIANT_SCORERS
selected_scorers = [
    all_scorers[key]
    for key in all_scorers
    if scorer_selections.get(key.lower(), False)
]

# Remove any scorers or output types that are not supported for the chosen organism.
unsupported_scorers = [
    scorer
    for scorer in selected_scorers
    if (
        organism.value
        not in variant_scorers.SUPPORTED_ORGANISMS[scorer.base_variant_scorer]
    )
    | (
        (scorer.requested_output == dna_client.OutputType.PROCAP)
        & (organism == dna_client.Organism.MUS_MUSCULUS)
    )
]
if len(unsupported_scorers) > 0:
  print(
      f'Excluding {unsupported_scorers} scorers as they are not supported for'
      f' {organism}.'
  )
  for unsupported_scorer in unsupported_scorers:
    selected_scorers.remove(unsupported_scorer)


# Score variants in the VCF file.
results = []

for i, vcf_row in tqdm(vcf.iterrows(), total=len(vcf)):
  variant = genome.Variant(
      chromosome=str(vcf_row.CHROM),
      position=int(vcf_row.POS),
      reference_bases=vcf_row.REF,
      alternate_bases=vcf_row.ALT,
      name=vcf_row.variant_id,
  )
  interval = variant.reference_interval.resize(sequence_length)

  variant_scores = dna_model.score_variant(
      interval=interval,
      variant=variant,
      variant_scorers=selected_scorers,
      organism=organism,
  )
  results.append(variant_scores)

df_scores = variant_scorers.tidy_scores(results)

if download_predictions:
  df_scores.to_csv('variant_scores.csv', index=False)
  files.download('variant_scores.csv')

df_scores

100%|██████████| 29/29 [01:07<00:00,  2.32s/it]


,variant_id,scored_interval,gene_id,gene_name,gene_type,gene_strand,junction_Start,junction_End,output_type,variant_scorer,...,biosample_type,biosample_life_stage,data_source,endedness,genetically_modified,transcription_factor,histone_mark,gtex_tissue,raw_score,quantile_score
0,chr5:87283216:C>T,chr5:86758928-87807504:.,None,None,None,None,None,None,ATAC,"CenterMaskScorer(requested_output=ATAC, width=...",...,primary_cell,adult,encode,paired,False,NaN,NaN,NaN,-0.014370,-0.536958
1,chr5:87283216:C>T,chr5:86758928-87807504:.,None,None,None,None,None,None,ATAC,"CenterMaskScorer(requested_output=ATAC, width=...",...,in_vitro_differentiated_cells,adult,encode,paired,False,NaN,NaN,NaN,-0.015421,-0.392960
2,chr5:87283216:C>T,chr5:86758928-87807504:.,None,None,None,None,None,None,ATAC,"CenterMaskScorer(requested_output=ATAC, width=...",...,primary_cell,adult,encode,paired,False,NaN,NaN,NaN,-0.004871,-0.248480
3,chr5:87283216:C>T,chr5:86758928-87807504:.,None,None,None,None,None,None,ATAC,"CenterMaskScorer(requested_output=ATAC, width=...",...,primary_cell,adult,encode,paired,False,NaN,NaN,NaN,-0.017655,-0.661785
4,chr5:87283216:C>T,chr5:86758928-87807504:.,None,None,None,None,None,None,ATAC,"CenterMaskScorer(requested_output=ATAC, width=...",...,primary_cell,adult,encode,paired,False,NaN,NaN,NaN,0.004197,0.114851
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
693164,chr20:32401544:G>T,chr20:31877256-32925832:.,ENSG00000171456,ASXL1,protein_coding,+,None,None,RNA_SEQ,PolyadenylationScorer(),...,tissue,embryonic,encode,single,False,NaN,NaN,,0.002362,0.551726
693165,chr20:32401544:G>T,chr20:31877256-32925832:.,ENSG00000171456,ASXL1,protein_coding,+,None,None,RNA_SEQ,PolyadenylationScorer(),...,tissue,embryonic,encode,single,False,NaN,NaN,,0.001233,0.248346
693166,chr20:32401544:G>T,chr20:31877256-32925832:.,ENSG00000171456,ASXL1,protein_coding,+,None,None,RNA_SEQ,PolyadenylationScorer(),...,tissue,embryonic,encode,single,False,NaN,NaN,,0.001233,0.257059
693167,chr20:32401544:G>T,chr20:31877256-32925832:.,ENSG00000171456,ASXL1,protein_coding,+,None,None,RNA_SEQ,PolyadenylationScorer(),...,tissue,embryonic,encode,single,False,NaN,NaN,,0.001621,0.354373


In [19]:
df_scores[df_scores['ontology_curie'] == 'UBERON:0001723'].to_csv('TJ_variant_scores.csv', index=False)
files.download('TJ_variant_scores.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>